In [1]:
import numpy as np

In [135]:
np.set_printoptions(formatter={'float_kind':"{:.2f}".format})

In [22]:
def s1(n):
    A = (
        + np.diag(np.ones(n)*5, 0)
        + np.diag(np.ones(n-1)*-1, -1)
        + np.diag(np.ones(n-1)*-1, 1)
    )
    b = np.ones(n)*3
    b[[0, -1]] = 4
    return A, b
s1(5)

(array([[ 5., -1.,  0.,  0.,  0.],
        [-1.,  5., -1.,  0.,  0.],
        [ 0., -1.,  5., -1.,  0.],
        [ 0.,  0., -1.,  5., -1.],
        [ 0.,  0.,  0., -1.,  5.]]),
 array([4., 3., 3., 3., 4.]))

In [24]:
def s2(n):
    A = (
        + np.diag(np.ones(n)*5, 0)
        + np.diag(np.ones(n-1)*-1, -1)
        + np.diag(np.ones(n-3)*-1, -3)
        + np.diag(np.ones(n-1)*-1, 1)
        + np.diag(np.ones(n-3)*-1, 3)
    )
    b = np.ones(n)
    b[[1, 2, -2, -3]] = 2
    b[[0, -1]] = 3
    return A, b
s2(7)

(array([[ 5., -1.,  0., -1.,  0.,  0.,  0.],
        [-1.,  5., -1.,  0., -1.,  0.,  0.],
        [ 0., -1.,  5., -1.,  0., -1.,  0.],
        [-1.,  0., -1.,  5., -1.,  0., -1.],
        [ 0., -1.,  0., -1.,  5., -1.,  0.],
        [ 0.,  0., -1.,  0., -1.,  5., -1.],
        [ 0.,  0.,  0., -1.,  0., -1.,  5.]]),
 array([3., 2., 2., 1., 2., 2., 3.]))

In [147]:
def s3(n):
    A = np.random.uniform(0, 1, size=(n, n))
    A += np.diag(np.ones(n))*(n+1)
    b = np.arange(1, n+1)
    return A, b
s3(10)

(array([[11.97, 0.33, 0.52, 0.55, 0.12, 0.30, 0.72, 0.70, 0.23, 0.93],
        [0.63, 11.91, 0.91, 0.43, 0.78, 0.17, 0.30, 0.37, 0.30, 0.47],
        [0.83, 0.22, 11.00, 0.07, 0.46, 0.24, 0.10, 0.61, 0.62, 0.84],
        [0.59, 0.51, 0.07, 11.02, 0.06, 0.83, 0.72, 0.73, 0.86, 0.65],
        [0.71, 0.15, 0.12, 0.89, 11.18, 0.56, 0.30, 0.47, 0.04, 0.03],
        [0.26, 0.75, 0.22, 0.01, 0.99, 11.43, 0.54, 0.63, 0.85, 0.32],
        [0.19, 0.32, 0.84, 0.67, 0.17, 0.67, 11.79, 0.45, 0.86, 0.20],
        [0.03, 0.84, 0.02, 0.33, 0.20, 0.58, 0.90, 11.66, 0.48, 0.62],
        [0.63, 0.34, 0.77, 0.74, 0.56, 0.90, 0.06, 0.33, 11.24, 0.35],
        [0.95, 0.59, 0.79, 0.69, 0.37, 0.89, 0.37, 0.44, 0.47, 11.50]]),
 array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]))

# Metoda Jacobi

In [96]:
def jacobi(A, b, iter=100, thresh=10**-10):
    assert len(A.shape) == 2
    assert len(b.shape) == 1
    assert A.shape[0] == A.shape[1] == b.shape[0]
    n = A.shape[0]
    
    d = np.diag(A)
    coef = np.concatenate([-b.reshape(-1, 1), A - np.diag(d)], 1) / -d.reshape(-1, 1)

    converged = False
    x = np.zeros(n)
    for i in range(iter):
        xp = x
        xe = np.concatenate([np.ones(1), xp])
        x = np.sum(coef * xe, 1)

        err = np.abs(np.mean(xp - x))
        if err < thresh:
            print(f"a convers in {i} iteratii")
            converged = True
            break
    if not converged:
        print("nu a convers")
    return x


n = 10

print("s1")
A, b = s1(n)
print(jacobi(A, b))
print(np.linalg.solve(A, b))

print("s2")
A, b = s2(n)
print(jacobi(A, b))
print(np.linalg.solve(A, b))

s1
a convers in 24 iteratii
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
s2
a convers in 56 iteratii
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


# Metoda Gauss-Seidel

In [97]:
def gauss_seidel(A, b, iter=100, thresh=10**-10):
    assert len(A.shape) == 2
    assert len(b.shape) == 1
    assert A.shape[0] == A.shape[1] == b.shape[0]
    n = A.shape[0]
    
    d = np.diag(A)
    coef = np.concatenate([-b.reshape(-1, 1), A - np.diag(d)], 1) / -d.reshape(-1, 1)

    converged = False
    x = np.zeros(n)
    for i in range(iter):
        xp = x
        xe = np.concatenate([np.ones(1), xp])
        for j in range(n):
            xe[j+1] = np.sum(coef[j] * xe)
        x = xe[1:]

        err = np.abs(np.mean(xp - x))
        if err < thresh:
            print(f"a convers in {i} iteratii")
            converged = True
            break
    if not converged:
        print("nu a convers")
    return x


n = 10

print("s1")
A, b = s1(n)
print(gauss_seidel(A, b))
print(np.linalg.solve(A, b))

print("s2")
A, b = s2(n)
print(gauss_seidel(A, b))
print(np.linalg.solve(A, b))

s1
a convers in 15 iteratii
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
s2
a convers in 31 iteratii
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


# Metoda SOR

In [159]:
def get_omega(A):
    D = np.diag(np.diag(A))
    L = np.tril(A) - D
    U = np.triu(A) - D
    
    T = np.matmul(np.linalg.inv(D), (L+U))
    spectral = np.max(np.abs(np.linalg.eigvals(T)))
    print(spectral)
    assert spectral < 1
    omega = 2 / (1 + np.sqrt(1 - spectral**2))
    return omega

In [194]:
def sor(A, b, iter=100, thresh=10**-10, omega=0.7):
    assert len(A.shape) == 2
    assert len(b.shape) == 1
    assert A.shape[0] == A.shape[1] == b.shape[0]
    n = A.shape[0]

    x = np.zeros(n)
    converged = False
    for i in range(iter):
        xp = x.copy()
        for j in range(n):
            x[j] = (
                + (1 - omega)*xp[j]
                + (omega / A[j, j]) * (
                    b[j] - np.sum(A[j, :j]*x[:j]) - np.sum(A[j, j+1:]*xp[j+1:])
                )
            )

        err = np.abs(np.mean(xp - x))
        if err < thresh:
            print(f"a convers in {i} iteratii")
            converged = True
            break
    if not converged:
        print("nu a convers")
    return x


n = 10

print("s1")
A, b = s1(n)
print(sor(A, b, omega=get_omega(A)))
print(np.linalg.solve(A, b))

print("s2")
A, b = s2(n)
print(sor(A, b, omega=get_omega(A)))
print(np.linalg.solve(A, b))

s1
0.3837971894457989
a convers in 16 iteratii
[1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00]
[1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00]
s2
0.676138620597146
a convers in 19 iteratii
[1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00]
[1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00]


In [181]:
A, b = s3(10)

print(np.linalg.solve(A, b))
print(jacobi(A, b))
print(gauss_seidel(A, b))
print(sor(A, b, omega=get_omega(A)))

[-0.10 0.02 0.17 0.20 0.27 0.36 0.47 0.56 0.64 0.73]
a convers in 25 iteratii
[-0.10 0.02 0.17 0.20 0.27 0.36 0.47 0.56 0.64 0.73]
a convers in 10 iteratii
[-0.10 0.02 0.17 0.20 0.27 0.36 0.47 0.56 0.64 0.73]
0.40780546453363703
a convers in 11 iteratii
[-0.10 0.02 0.17 0.20 0.27 0.36 0.47 0.56 0.64 0.73]
